In [2]:
import numpy as np
import numpy.random as npr
import cvxpy as cp
import tensorflow as tf
from cvxpylayers.tensorflow import CvxpyLayer

In [63]:
n, m, p = 20, 10, 10

# construct an QP
_x = npr.rand(n)
Q = npr.rand(n,n) 
Q = np.dot(Q.T, Q) # ensure PSD
q = npr.rand(n)
__G = npr.rand(m,n)
h = __G@_x + npr.rand(m)
A = npr.rand(p,n)
b = A@_x

x = cp.Variable(n)    # define variable
G = cp.Parameter((m,n)) # define params

constraints = [G@x <= h, A@x == b]
objective = cp.Minimize(0.5*cp.quad_form(x, Q) + q.T @ x)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

In [64]:
cvxpylayer = CvxpyLayer(problem, parameters=[G], variables=[x])

_G = tf.Variable(__G)

with tf.GradientTape() as tape:
  # solve the problem
  solution, = cvxpylayer(_G)
    
  summed_solution = tf.math.reduce_sum(solution)

# compute the gradient of the summed solution with respect to params
gradh = tape.gradient(summed_solution, [_G])

In [65]:
solution

<tf.Tensor: shape=(20,), dtype=float64, numpy=
array([-2.46594574,  3.79540751, -0.42787383, -5.40518841,  4.16340034,
       -3.21156128,  3.57073269,  4.04120549, -1.80894942,  1.47101546,
       -3.34872721,  2.1831713 ,  0.96126981, -2.85702765, -0.48223582,
       -7.75541162, -0.14466602, -1.04196598,  9.21881514,  1.48823202])>

In [77]:
collective = np.array([Q,q,__G,h,A,b,gradh[0].numpy()])